In [10]:
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
import re
import time
import csv
import os

In [11]:
def getweather(siteurl):
        #########开始记录PM10等污染物信息
        resp=urlopen(siteurl)
        ##'https://tianqi.moji.com/pm/china/beijing/xicheng-district'
        soup=BeautifulSoup(resp,'html.parser')
        time=soup.find('b')
        time=time.string[5:]
        ###print(time)
        AQI=soup.find('em',id='aqi_value').string
        #clearfix=soup.find('ul',class_='clearfix')
        ###print('AQI：'+str(AQI))
        #print(clearfix)
        
        weatherinfo=[time,AQI]#天气信息
        
        weather=soup.find_all('li')
        PM10,PM2_5,NO_2,SO_2,O_3,CO=0,0,0,0,0,0
        clearfix=[PM10,PM2_5,NO_2,SO_2,O_3,CO]
        for i in range(6):
            clearfix[i]=weather[i].span.string
            weatherinfo.append(clearfix[i])
        ###print(clearfix)
        
        ########开始记录温度湿度风
        resp=urlopen('https://tianqi.moji.com/today'+siteurl[26:])
        soup=BeautifulSoup(resp,'html.parser')
        today=soup.find('div',class_='info clearfix')
        temperature=today.em.string
        weather=today.b.string
        today=soup.find('div',class_='about clearfix')
        humidity=today.span.string[2:]
        wind=today.em.string
        listtoday=[temperature,humidity,wind,weather]
        weatherinfo.extend(listtoday)
        
        return weatherinfo

In [12]:
def writeData(weatherinfo,sitename):
    path = 'weather-china/'  # 设置路径，也可设为相对路径

    # 判断目录是否存在
    if not os.path.exists(os.path.split(path)[0]):
        # 目录不存在创建，makedirs可以创建多级目录
        os.makedirs(os.path.split(path)[0])
    try:
        # 保存数据到文件
        with open(path+sitename+'.csv','a',encoding='utf-8-sig',newline='') as f:
            writer=csv.writer(f,dialect='excel')
            writer.writerow(weatherinfo)
        ##print('保存成功')
    except Exception as e:
        print('保存失败', e)

In [13]:
def getsites():
        #########爬取网页地址和地点
        resp=urlopen('https://tianqi.moji.com/pm/china')
        soup=BeautifulSoup(resp,'html.parser')
        site=soup.find_all('li')
        sitename,siteurl=[],[]
        for i in range(31):
            sitename.append(site[i].a.string)
            siteurl.append(site[i].a.get('href'))
        sitenames,siteurls=[],[]
        for i in range(len(sitename)):  
                resp=urlopen('https://tianqi.moji.com'+siteurl[i])
                soup=BeautifulSoup(resp,'html.parser')
                shenghui=siteurl[i].split('/')
                shenghui=shenghui[3]
                site=soup.find_all(href=re.compile(shenghui+'/'))
                for _ in site:
                    sitenames.append(sitename[i]+_.string)
                    siteurls.append(_.get('href'))
    
        return sitenames,siteurls

In [14]:
def main():
    #模拟成浏览器 
    headers=("User-Agent","Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36") 
    opener = urllib.request.build_opener() 
    opener.addheaders=[headers] 
    #将opener安装为全局 
    urllib.request.install_opener(opener)
    ##############################################
    
    sitenames,siteurls=getsites()
    for i in range(len(sitenames)):
        try:
            weatherinfo=getweather(siteurls[i])
            writeData(weatherinfo,sitenames[i])
        except urllib.error.URLError as e: 
            print("error"+sitenames[i])
            continue
        except Exception as e: 
            print("error"+sitenames[i])
            continue

        if i%100==0:
            print("100----------")

    print("done")

In [ ]:
if __name__=='__main__':
        while True: 
                print("开始工作！") 
                start = time.time()
                main()
                #每一小时执行一次 
                print("休息中……")
                end = time.time()
                if end-start>3600:
                    continue
                else:
                    time.sleep(3600-(end-start))

开始工作！
100----------
100----------
100----------
100----------
100----------
100----------
error甘肃瓜州县
error甘肃玛曲县
error甘肃夏河县
error甘肃舟曲县
error甘肃临潭县
error甘肃碌曲县
error甘肃文县
error甘肃两当县
error甘肃西和县
error甘肃成县
error甘肃合作市
error甘肃武都区
100----------
error甘肃康县
error甘肃宕昌县
error甘肃礼县
error甘肃卓尼县
error甘肃徽县
error甘肃迭部县
error甘肃甘南藏族自治州
error广东电白区
100----------
100----------
100----------
